In [13]:
import numpy as np
import pandas as pd
from itertools import product

from pyomo.environ import *

# Define data
T = 7
alpha = 0.1

sources_data = {
    "s1": [0.06, 0, 0, [10, 10, 10, 0, 0, 0, 0]],
    "s2": [0.26, 0, 0, [30, 30, 30, 0, 0, 0, 0]]
}

demands_data = {
    "d1": [0, 0.16, 0, 0, [0, 0, 15, 15, 15, 15], 2],
    "d2": [0, 0.10, 0, 0, [0, 0, 15, 15, 15, 15], 1]
}

blenders_data = {
    "b1": 30,
    "b2": 30,
    "b3": 30,
    "b4": 30,
    "b5": 20,
    "b6": 20,
    "b7": 20,
    "b8": 20
}

sources = list(sources_data.keys())
demands = list(demands_data.keys())
blenders = list(blenders_data.keys())

s_inv_lb = {s: sources_data[s][1] for s in sources}
s_inv_ub = {s: sources_data[s][2] for s in sources}
s_amounts = {s: sources_data[s][3] for s in sources}

d_quals_lb = {d: demands_data[d][0] for d in demands}
d_quals_ub = {d: demands_data[d][1] for d in demands}
d_inv_lb = {d: demands_data[d][2] for d in demands}
d_inv_ub = {d: demands_data[d][3] for d in demands}
d_amounts = {d: demands_data[d][4] for d in demands}
betaT_d = {d: demands_data[d][5] for d in demands}

b_inv_ub = {b: blenders_data[b] for b in blenders}

In [14]:
# Model
model = ConcreteModel()

# Sets
model.sources = Set(initialize=sources)
model.demands = Set(initialize=demands)
model.blenders = Set(initialize=blenders)

In [15]:
blenders

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8']

In [4]:
# Parameters
model.s_inv_lb = Param(model.sources, initialize=s_inv_lb)
model.s_inv_ub = Param(model.sources, initialize=s_inv_ub)
model.s_amounts = Param(model.sources, initialize=s_amounts)
model.d_quals_lb = Param(model.demands, initialize=d_quals_lb)
model.d_quals_ub = Param(model.demands, initialize=d_quals_ub)
model.d_inv_lb = Param(model.demands, initialize=d_inv_lb)
model.d_inv_ub = Param(model.demands, initialize=d_inv_ub)
model.d_amounts = Param(model.demands, initialize=d_amounts)
model.betaT_d = Param(model.demands, initialize=betaT_d)
model.b_inv_ub = Param(model.blenders, initialize=b_inv_ub)

# Decision variables
model.x = Var(model.sources, model.blenders, domain=NonNegativeReals)

'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'Any'. The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend the
domain of this Paramto be 'Any', you can suppress this warning by explicitly
specifyin

In [5]:
# Objective function
def obj_rule(model):
    return sum(model.x[s, b] * model.s_amounts[s, b] for s in model.sources for b in model.blenders)

model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
def inv_balance_rule(model, s):
    return (model.s_inv_lb[s], sum(model.x[s, b] for b in model.blenders), model.s_inv_ub[s])

model.inv_balance_con = Constraint(model.sources, rule=inv_balance_rule)

def demand_rule(model, d):
    return (model.d_quals_lb[d], sum(model.x[b, d] * model.d_quals_ub[d] for b in model.blenders) / model.d_amounts[d], model.d_quals_ub[d])

model.demand_con = Constraint(model.demands, rule=demand_rule)

def blender_inv_rule(model, b):
    return (0, sum(model.x[b, d] for d in model.demands), model.b_inv_ub[b])

model.blender_inv_con = Constraint(model.blenders, rule=blender_inv_rule)

ERROR: Rule failed when generating expression for Objective obj with index
None: KeyError: "Index '('s1', 's1')' is not valid for indexed component
's_amounts'"
ERROR: Constructing component 'obj' from data=None failed:
        KeyError: "Index '('s1', 's1')' is not valid for indexed component
        's_amounts'"


KeyError: "Index '('s1', 's1')' is not valid for indexed component 's_amounts'"

In [ ]:
# Solve the model
solver = SolverFactory('gurobi')
solver.solve(model)

In [ ]:

# Print results
for s in model.sources:
    for b in model.blenders:
        print(f"Amount from {s} to {b}: {model.x[s, b].value}")

print("Objective Value:", model.obj())
